In [1]:
from SagasuSubs import utils,models,patch
from pathlib import Path

patch.patch()

In [2]:
workdir = Path('.') / 'subtitles' / 'subs_list'

files = [*utils.iterate_files(workdir,'.ass')]

files[:5]

[PosixPath('subtitles/subs_list/animation/1970/(1970.4.1)明日之丈 第1季/BD/冷番补完字幕组/[冷番补完][明日之丈][あしたのジョー][01][480P][简体外挂].ass'),
 PosixPath('subtitles/subs_list/animation/1970/(1970.4.1)明日之丈 第1季/BD/冷番补完字幕组/[冷番补完][明日之丈][あしたのジョー][02][480P][简体外挂].ass'),
 PosixPath('subtitles/subs_list/animation/1970/(1970.4.1)明日之丈 第1季/BD/冷番补完字幕组/[冷番补完][明日之丈][あしたのジョー][03][480P][简体外挂].ass'),
 PosixPath('subtitles/subs_list/animation/1970/(1970.4.1)明日之丈 第1季/BD/冷番补完字幕组/[冷番补完][明日之丈][あしたのジョー][04][480P][简体外挂].ass'),
 PosixPath('subtitles/subs_list/animation/1970/(1970.4.1)明日之丈 第1季/BD/冷番补完字幕组/[冷番补完][明日之丈][あしたのジョー][05][480P][简体外挂].ass')]

In [3]:
from tqdm.notebook import tqdm
from traceback import format_exc,print_exc
import re

from SagasuSubs.database import FileCrud,DialogCrud,dto

DB_PATH = Path('.') / 'db.sqlite3'
file_crud = FileCrud(DB_PATH)
dialog_crud =DialogCrud(DB_PATH)


try:
    file_crud.create_table()
    dialog_crud.create_table()
except Exception as e:
    print_exc(-1)

PATH_REGEX = re.compile(
    r'^.+/(?P<year>\d+?)/\((?P<date>[0-9.]+?)\)(?P<name>.+?)/(?P<child_path>.+)$'
)

Traceback (most recent call last):
  File "/home/mix/Documents/SagasuSubs/.venv/lib/python3.9/site-packages/sqlalchemy/engine/default.py", line 717, in do_execute
    cursor.execute(statement, parameters)
sqlite3.OperationalError: table files already exists

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/mix/Documents/SagasuSubs/.venv/lib/python3.9/site-packages/sqlalchemy/engine/default.py", line 717, in do_execute
    cursor.execute(statement, parameters)
sqlalchemy.exc.OperationalError: (sqlite3.OperationalError) table files already exists
[SQL: 
CREATE TABLE files (
	id INTEGER NOT NULL, 
	filename VARCHAR(1024) NOT NULL, 
	sha1 VARCHAR(40) NOT NULL, 
	path VARCHAR NOT NULL, 
	series_name VARCHAR, 
	series_id INTEGER, 
	episode_name VARCHAR, 
	episode_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(series_id) REFERENCES series (id), 
	FOREIGN KEY(episode_id) REFERENCES episode (id)
)

]
(Background on this error 

In [5]:
progress = tqdm(files)

skipped = 0

for count,file in enumerate(progress):
    progress.set_description(f'{file=} ,{skipped=}, {skipped/(count+1)=:.3%}')
    
    try:
        data = models.SubtitlePersist.from_file(file,format='ass',exclude_fx=True)
    except Exception:
        progress.write(
            '\n-->\t'.join(
                [f'Error occurred during processing file {file}:'] +
                format_exc(-3).splitlines()
            )
        )
        skipped += 1
        continue

    if not (matched := PATH_REGEX.match(str(file))):
        progress.write(f'{data.path=} does not have valid regex data, skip')
        skipped += 1
        continue

    if existed_file := file_crud.read_by_sha1(data.sha1):
        skipped += 1
        continue
        
    file_model = file_crud.create(dto.FileCreate(
        filename=data.filename,
        sha1=data.sha1,
        path=str(data.path),
        series_name=matched.group('name')
        )
    )

    dialog_crud.create_bulk(
            [
                dto.DialogCreate(content=dialog.content,begin=dialog.begin,end=dialog.end,file_id=file_model.id)
                for dialog in data.dialogs
            ]
        )
        

  0%|          | 0/10191 [00:00<?, ?it/s]

/home/mix/Documents/SagasuSubs/SagasuSubs/patch.py:59: UserWarning: Failed to parse field f='start',v='0:00:-1.00' due to AttributeError("'NoneType' object has no attribute 'groups'"), fallback to None
  warn(f"Failed to parse field {f=!r},{v=!r} due to {e!r}, fallback to None")
/home/mix/Documents/SagasuSubs/SagasuSubs/patch.py:59: UserWarning: Failed to parse field f='end',v='0:00:-1.00' due to AttributeError("'NoneType' object has no attribute 'groups'"), fallback to None
  warn(f"Failed to parse field {f=!r},{v=!r} due to {e!r}, fallback to None")
/home/mix/Documents/SagasuSubs/SagasuSubs/patch.py:59: UserWarning: Failed to parse field f='start',v='0:00:00.-13' due to AttributeError("'NoneType' object has no attribute 'groups'"), fallback to None
  warn(f"Failed to parse field {f=!r},{v=!r} due to {e!r}, fallback to None")
/home/mix/Documents/SagasuSubs/SagasuSubs/patch.py:59: UserWarning: Failed to parse field f='end',v='0:00:00.-13' due to AttributeError("'NoneType' object has no

Error occurred during processing file subtitles/subs_list/animation/2019/(2019.10.5)战×恋/BD/[MMSUB] Val x Love_01-12_END/SC/[MMSUB] Val x Love - 07 [WebRip 1080p AVC-8bit AAC ASSx2].SC.ass:
-->	Traceback (most recent call last):
-->	  File "/tmp/ipykernel_14687/417691176.py", line 9, in <module>
-->	    data = models.SubtitlePersist.from_file(file,format='ass',exclude_fx=True)
-->	  File "/home/mix/Documents/SagasuSubs/SagasuSubs/models.py", line 58, in from_file
-->	    DialogContent(
-->	  File "pydantic/main.py", line 406, in pydantic.main.BaseModel.__init__
-->	pydantic.error_wrappers.ValidationError: 2 validation errors for DialogContent
-->	begin
-->	  none is not an allowed value (type=type_error.none.not_allowed)
-->	end
-->	  none is not an allowed value (type=type_error.none.not_allowed)
Error occurred during processing file subtitles/subs_list/animation/2019/(2019.10.5)战×恋/BD/[MMSUB] Val x Love_01-12_END/SC/[MMSUB] Val x Love - 08 [WebRip 1080p AVC-8bit AAC ASSx2].SC.ass:
-->

/home/mix/Documents/SagasuSubs/SagasuSubs/patch.py:59: UserWarning: Failed to parse field f='layer',v='Marked=0' due to ValueError("invalid literal for int() with base 10: 'Marked=0'"), fallback to None
  warn(f"Failed to parse field {f=!r},{v=!r} due to {e!r}, fallback to None")
/home/mix/Documents/SagasuSubs/SagasuSubs/patch.py:59: UserWarning: Failed to parse field f='marginl',v='{\\fn@方正准雅宋_GBK\\fs40\\blur3\\frz270\\pos(740' due to ValueError("invalid literal for int() with base 10: '{\\\\fn@方正准雅宋_GBK\\\\fs40\\\\blur3\\\\frz270\\\\pos(740'"), fallback to None
  warn(f"Failed to parse field {f=!r},{v=!r} due to {e!r}, fallback to None")
/home/mix/Documents/SagasuSubs/SagasuSubs/patch.py:59: UserWarning: Failed to parse field f='marginr',v='390)}五彩斑斕少女心' due to ValueError("invalid literal for int() with base 10: '390)}五彩斑斕少女心'"), fallback to None
  warn(f"Failed to parse field {f=!r},{v=!r} due to {e!r}, fallback to None")
/home/mix/Documents/SagasuSubs/SagasuSubs/patch.py:59: UserWa